## Project 4
## Submitted by Roshan Lulu

### Step 1 - Get the Job Posting data from the website

- Identify the Job titles to scrap
- Identify the element/child relationships and get the attributes
- Baby step - Get 1 job posting scraped


### A. Import Libraries

In [11]:
# import libraries
import scrapy
from scrapy.http import Request
from scrapy import Selector
import requests
import pandas as pd


### B. GET THE PAGE LINKS FOR THE JOBS

In [12]:
# # deinfe the serach urls
# urls = [{'page':1,'type': 'data-analyst', 'url':'https://www.seek.com.au/Data-Analyst-jobs/in-All-Sydney-NSW'},
#                 {'page':1,'type': 'data-science', 'url':'https://www.seek.com.au/Data-Science-jobs/in-All-Sydney-NSW'},
#                  {'page':1, 'type': 'business-intelligence', 'url':'https://www.seek.com.au/Business-Intelligence-jobs/in-All-Sydney-NSW'}]

job_desc_i_want = ['Data-Science', 'Data-Analyst', 'Business-Intelligence']
where_is_the_job = '-jobs/in-All-Sydney-NSW'
search_urls = []

for index in range(0, len(job_desc_i_want)):
    job_dict = {}
    job_dict['type'] = job_desc_i_want[index]
    job_dict['page'] = 1
    job_dict['url'] = 'https://www.seek.com.au/' + job_desc_i_want[index] + where_is_the_job
    search_urls.append(job_dict)

search_urls

[{'page': 1,
  'type': 'Data-Science',
  'url': 'https://www.seek.com.au/Data-Science-jobs/in-All-Sydney-NSW'},
 {'page': 1,
  'type': 'Data-Analyst',
  'url': 'https://www.seek.com.au/Data-Analyst-jobs/in-All-Sydney-NSW'},
 {'page': 1,
  'type': 'Business-Intelligence',
  'url': 'https://www.seek.com.au/Business-Intelligence-jobs/in-All-Sydney-NSW'}]

In [13]:
pagelinks_df = pd.DataFrame(columns = ('Job_Desc', 'Page_Num', 'url_link'))

In [14]:
# Define the start request function
def get_all_page_links_per_job(url, job_desc, page_num=1):
    # Get the html response from the first page url
    response = requests.get(url)
    # get the link to the next page
    next_page_anchor  =  Selector(text=response.text).xpath("//a[@data-automation='page-next']/@href").extract()
    # check if there is a next page
    if len(next_page_anchor) > 0:
        # Save the current url details before finding the next one
        pagelinks_df.loc[len(pagelinks_df),:] = [job_desc, page_num, url]
        # Start searching for next page link
        # Recursively call to get the links in next anchor page
        page_num += 1
        next_page_anchor = "https://www.seek.com.au" + next_page_anchor[0]
        get_all_page_links_per_job(next_page_anchor, job_desc, page_num)
    else:
        pass

In [15]:
def parse_all_first_pages(urls=search_urls):
    for url in urls:
        get_all_page_links_per_job(url['url'], url['type']) 

#Parse all the required job search results and get the links to the job postings
parse_all_first_pages()

In [16]:
# Display the dataframe obtained
pagelinks_df.head()

,Job_Desc,Page_Num,url_link
0,Data-Science,1,https://www.seek.com.au/Data-Science-jobs/in-A...
1,Data-Science,2,https://www.seek.com.au/Data-Science-jobs/in-A...
2,Data-Science,3,https://www.seek.com.au/Data-Science-jobs/in-A...
3,Data-Science,4,https://www.seek.com.au/Data-Science-jobs/in-A...
4,Data-Science,5,https://www.seek.com.au/Data-Science-jobs/in-A...


In [26]:
# Get all the job posting links from each page
list_of_posts = []
for index in range(0, len(pagelinks_df)):
    # Get the response text from the url
    url = pagelinks_df.iloc[index, 2]
    response = requests.get(url)
    # Parse the response text
    # parse_job_posting(response)
    post = Selector(text=response.text).xpath('//a[@data-automation="jobTitle"]/@href').extract()
    list_of_posts.append(post)

pagelinks_df['Job_Posts'] = list_of_posts

### THE TEXT PARSE STARTS NOW !

In [ ]:
### C. Go through each page link and get the 
# Description, Job_Title, Salary, Location, Work_Type, Classification,
# DatePosted, Postedby

In [33]:
# Define the start request function
titles = []
jd1 = []
jd2 = []
for index in range(0, 5):
    # Get the response text from the url
    urls = pagelinks_df.iloc[index, 3]
    for url in urls:
        response = requests.get('https://www.seek.com.au' + url)
        # Parse the response text
        # parse_job_posting(response)
#         main_path = '//div[@data-automation="joblisting-index"]/div[@class="mod-job-details"]/ul/li/div/span[@itemprop="addressRegion"]/span/text()'                 
        job_title = Selector(text=response.text).xpath('//div[@data-automation="joblisting-index"]/div[@class="grid_6"]/h1/text()').extract()
        job_desc_1 = Selector(text=response.text).xpath('//div[@class="templatetext"]/p//text()').extract()
        job_desc_2 = Selector(text=response.text).xpath('//div[@class="templatetext"]/ul/li//text()').extract()
        print(job_title)

['Senior Manager - Data Science']
['Head of Data Science - ML & AI']
['Head of Data Science & Analytics | Strategic role | Innovative Tech Product Co']
['Head of Data Science - Unique Career Opportunity -Innovative Technology Company']
['Head of Data Science &amp; Products']
['Head of Data Science &amp; Products']
['Head of Data Science &amp; Products']
['Computational Statistician | Data Science and Research Software Business']
['Junior Data Scientist (Modelling, Insights & Data Science)']
['DATA SCIENCE EXPERT ']
['Business Analyst- Data Analytics / Data Science ']
['Data Scientist']
['JAPANESE-SPEAKING DATA SCIENTIST - REMOTE']
['Senior Data Scientist | Strategic Position | Innovative & Dynamic Organisation']
['Junior Software Engineer']
['Lead Data Scientist']
['Data Analyst / Data Scientist']
['Senior Client Consultant - Financial Services']
['Forensic Data Analytics']
['Principal Consultant - Analytics']
['Senior Data Scientist']
['Senior Data Scientist']
['Data Engineer/Scientis